In [13]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [14]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm

In [15]:
# ----------------- Parameters

# The file to consider
file = './file_with_optimal_val/example.data'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = False

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 300

# Number of iteration stop criterion
it_max = 20

# Parameters of the first neural network
nb_hidden_layers1 = 2
nb_neurons1 = [3, 2, 1]

# Parameters of the second neural network
nb_hidden_layers2 = 1
nb_neurons2 = [3, 1]


# The number of solutions that we will have in the solver after the first iteration
k_k = 10

In [16]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, tasks = fm.initialize_solver(data, n, m, duration)

model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)

# ------------ Solve the model
print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)

# ------------ Display the result
# fm.display_solution(msol, display_sol)
print("Model solved !")

# ---------------- Interaction with the user
print("\n--------Interaction with the user...---------")
print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model

list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# Vector of the start time of each task of each preference
starts = user.start_pref(n, m, display_start)

# Matrix of the start time of each task of each preference
matrix = user.matrix_pref(n, m, display_matrix)

# Testing the order of preferences and the differences between solutions
fm.test(n, m, user)

print("list layers : ",list_layers)

sol_layers = fm.list_list_list_start_of_tasks(n, m, list_layers)


--------Main program is loading...---------

Creating the model variables...
Model tasks_by_jobs created !

Solver initialized !

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !

Solving the model...
Model solved !

--------Interaction with the user...---------

Creating the user...
User created !

Classing solutions...
Le min de list_obj est 348
Le max de list_obj est 793
Solutions classed !

Creating preferences...
Preferences created !

Testing order of preferences...
	L'ordre des préférences n'est pas cohérente

Testing differences between solutions...
	Toutes les solutions sont différentes
list layers :  [[<docplex.cp.solution.CpoSolveResult object at 0x000001F4468FF3C0>, <docplex.cp.solution.CpoSolveResult object at 0x000001F44AEA9EC0>, <docplex.cp.solution.CpoSolveResult object at 0x000001F447D16AC0>, <docplex.cp.solution.CpoSolveResult object at 0x000001F44CA68040>, <

In [17]:
# a = 1 means that the order of the sol is 0 
# a = 0 means that the order of the sol is not 0

def activation_function(model, solver, input, weights, nb_hidden_layers, nb_neurons, id_layer = 0):

    if id_layer == 0:
        sum_input = sum(input)
    else:
        sum_input = len(input)
    
    # S = [model.integer_var(min=-2*optimalval, max=2*optimalval, name="S{}{}".format(id_layer, j)) for j in range(nb_neurons[id_layer])]
    S = [model.integer_var(min=-sum_input, max=sum_input, name="S{}{}".format(id_layer, j)) for j in range(nb_neurons[id_layer])]
    a = [model.binary_var(name="a{}{}".format(id_layer, j)) for j in range(nb_neurons[id_layer])]
    # solver.add_variable_model(model, S)
    # solver.add_variable_model(model, a)
    # S = 0

    output = []
    for j in range(nb_neurons[id_layer]):
        
        # S =  sum(input[k] * weights[id_layer][j][k] for k in range(len(input)))
        # a = (S>0)

        # print("S = ", S)
        # print("a = ", a)

        solver.add_constraint(model, S[j] == sum(input[k] * weights[id_layer][j][k] for k in range(len(input))))
        solver.add_constraint(model, a[j] == (S[j] > 0))
        model.add(S[j])
        model.add(a[j])
        # solver.add_constraint(model, a[j].equals(1))
        
        if id_layer == nb_hidden_layers: 
            return a[j]
            # return a
        else: 
            output.append(a[j])
            # output.append(a)

    return activation_function(model, solver, output, weights, nb_hidden_layers, nb_neurons, id_layer + 1)

In [18]:
####################################################################
#### NOUVELLES SOLUTIONS ET NOUVEAUX CLUSTERING A CHAQUE ITERATION
####################################################################

# --------- Call Solver constructor in Solver.py and create the variables of the model
# model, solver, tasks = fm.initialize_solver(data, n, m, duration)

solver = Solver(data)
model = CpoModel() 

# --------- Create the model variables ---------
# --------- Create the model variables 
print("\nCreating the model variables...")
# Create task interval variables and add them to the solver

tasks = [[None for j in range(m)] for i in range(n)]
for i in range(n):
    for j in range(m):
        # print(duration[i][j])
        tasks[i][j] = model.integer_var(min=0, max=2*optimalval, name="T{}-{}".format(i,j))

# ------------ Add tasks to the solver
solver.add_variable(tasks)
print("Model tasks_by_jobs created !")


print("\nSolver initialized !")

# ---------------- Interaction with the neural network

# Create the weights of the neural network
weights01 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons1[0])]]
weights1 = weights01 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons1[i-1])] for j in range(nb_neurons1[i])] for i in range(1, nb_hidden_layers1 + 1)]

print("\n len of weights1 :", len(weights1))
for i in range(len(weights1)):
    print("\n  len of weights1[i] :", len(weights1[i]))
    for j in range(len(weights1[i])):
        print("\n   len of weights1[i][j] :", len(weights1[i][j]))
        for k in range(len(weights1[i][j])):
            model.add(weights1[i][j][k])
            
# solver.add_variable_model(model, weights1)

# weights02 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons2[0])]]
# weights2 = weights02 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons2[i-1])] for j in range(nb_neurons2[i])] for i in range(1, nb_hidden_layers2 + 1)]
# solver.add_variable_model(model, weights2)

# Train the neural network

for order in range(len(sol_layers)):
    for sol in sol_layers[order]:

        output_NN1 = activation_function(model, solver, sol, weights1, nb_hidden_layers1, nb_neurons1)
        # output_NN2 = activation_function(model, solver, sol, weights2, nb_hidden_layers2, nb_neurons2)

        if order == 0:
            solver.add_constraint(model, output_NN1 == 1)
        #     # solver.add_constraint(model, output_NN2 == 1)
        # else:
        #     solver.add_constraint(model, output_NN1 == 0)
        #     # solver.add_constraint(model, output_NN2 == 0)


# Compare the result of 2 different neural networks for the same solution

# tasks_starts = []
# for i in range(n):
#     for j in range(m):
#         tasks_starts.append(model.start_of(tasks[i][j]))

# outputvar_NN1 = activation_function(model, solver, tasks[0], weights1, nb_hidden_layers1, nb_neurons1)
# outputvar_NN2 = activation_function(model, solver, tasks[0], weights2, nb_hidden_layers2, nb_neurons2)
# solver.add_constraint(model, outputvar_NN1 != outputvar_NN2)
# solver.add_constraint(model, logical_or(outputvar_NN1 == 1, outputvar_NN2 == 1))


# ------------ Solve the model
print("\nSolving the model...")

# list = model.get_all_variables()
# for i in range(len(list)):
#     print(list[i].get_domain())
# print(model.get_statistics())


# msol = model.start_search(TimeLimit=60, LogVerbosity="Quiet")
# for sol in msol:
#     sol.write()

msol = model.solve(TimeLimit=60, LogVerbosity="Quiet")
print(msol.get_fail_status())
print(msol.get_search_status())
print(msol.get_objective_value())
print(msol.get_solution())
# msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)
# print("The number of solutions generated is :",nb_solution)

# ------------ Display the result
fm.display_solution(msol, display_sol)
# print("Model solved !")


Creating the model variables...
Model tasks_by_jobs created !

Solver initialized !

 len of weights1 : 3

  len of weights1[i] : 3

   len of weights1[i][j] : 36

   len of weights1[i][j] : 36

   len of weights1[i][j] : 36

  len of weights1[i] : 2

   len of weights1[i][j] : 3

   len of weights1[i][j] : 3

  len of weights1[i] : 1

   len of weights1[i][j] : 2

Solving the model...
<ipython-input-18-fc4407248cfe>:41(stream:10:12): Warning: Unused integer variable 'w0-0-0'.
                                                             "w0-0-0" = intVar(-1..1)
<ipython-input-18-fc4407248cfe>:41(stream:11:12): Warning: Unused integer variable 'w0-0-1'.
                                                             "w0-0-1" = intVar(-1..1)
<ipython-input-18-fc4407248cfe>:41(stream:12:12): Warning: Unused integer variable 'w0-0-2'.
                                                             "w0-0-2" = intVar(-1..1)
<ipython-input-18-fc4407248cfe>:41(stream:13:12): Warning: Unused integer